# InSAR workflow

This notebook contain an example on how to execute the InSAR workflow on a local machine with SNAP installed. This workflow can produce geocoded InSAR coherence and flattened + topographic corrected + multilooked + filtered wrapped and unwrapped interferogram

In [1]:
import urllib
import json
from IPython.display import JSON

from datetime import datetime

import glob
import os
import subprocess
import shutil
import subprocess
import re

## Input definition

In [2]:
burstId = 62083
sub_swath = 'IW3'
InSAR_pairs = [
    '20190502_20190526',
    '20190420_20190502',
    '20190502_20190607',
    '20190327_20190502',
    '20190408_20190502',
    '20190408_20190420',
    '20190315_20190408',
    '20190514_20190619',
    '20190514_20190526',
    '20190420_20190526',
    '20190303_20190408',
    '20190408_20190514',
    '20190327_20190420',
    '20190502_20190514',
    '20190303_20190327',
    '20190327_20190408',
    '20190420_20190514',
    '20190315_20190420',
    '20190303_20190315',
    '20190514_20190607',
    '20190315_20190327'
]

# Coherence window size
cohWinRg, cohWinAz = (11, 3)

# Multillok parameters
nRgLooks, nAzLooks = (4, 1)

# Area of interest
aoi = 'POLYGON((18.02 -33.07, 18.02 -32.96, 18.3 -32.96, 18.3 -33.07, 18.02 -33.07))'

# Output folder
output_folder = '/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/mintpy/'
if not os.path.exists(output_folder):
    output_folder = os.path.abspath("output")
print("output_folder: " + str(output_folder))

output_folder: /home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/mintpy/


In [3]:
graph_interferogram = '../graphs/interferogram_sarGeometry.xml'
graph_snaphu_export = '../graphs/snaphu_export.xml'
graph_geocode = '../graphs/geocode_snaphuInterferogram_WGS84.xml'
graph_subset = '../graphs/subset.xml'

In [4]:
if os.path.exists("../CDSE_SECRET"):
    with open("../CDSE_SECRET", "r") as file:
        lines = file.readlines()
    CDSE_ACCESS_KEY = lines[0].strip().split(": ")[1]
    CDSE_SECRET_KEY = lines[1].strip().split(": ")[1]
else:
    # get from environment vatiables:
    CDSE_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY_ID", None)
    CDSE_SECRET_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY", None)

## Burst download

In [5]:
date_list = [datetime.strptime(date, "%Y%m%d") for date in list(set('_'.join(InSAR_pairs).split('_')))]

for date in date_list:

    https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
    f"ContentDate/Start ge {date.strftime('%Y-%m-%d')}T00:00:00.000Z and ContentDate/Start le {date.strftime('%Y-%m-%d')}T23:59:59.000Z and "
    f"PolarisationChannels eq 'VV' and "
    f"BurstId eq {burstId} and "
    f"SwathIdentifier eq '{sub_swath}'"
    )

    with urllib.request.urlopen(https_request) as response:
        content = response.read().decode()
    bursts = json.loads(content)
    ParentProductName = bursts['value'][0]['ParentProductName']

    burst_extract_cmd = (
        f"docker run -v {output_folder}:/home/ubuntu "
        f"-e AWS_ACCESS_KEY_ID={CDSE_ACCESS_KEY} -e AWS_SECRET_ACCESS_KEY={CDSE_SECRET_KEY} "
        f"cdse_utilities sentinel1_burst_extractor.sh -o /home/ubuntu "
        f"-n {ParentProductName} -p vv -s {sub_swath.lower()} -r {burstId}"
    )

    #print(burst_extract_cmd)
    os.system(burst_extract_cmd)
    

Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 24480, 1506
0...10...20...30...40...50...60...70...80...90...100 - done.


## Sentinel-1 InSAR processing

In [8]:
saveDEM = True
for pair in InSAR_pairs:

    # Create the interferogram
    prm_filename = glob.glob(os.path.join(output_folder, f'*{pair.split("_")[0]}*', 'manifest.safe'))[0]
    sec_filename = glob.glob(os.path.join(output_folder, f'*{pair.split("_")[1]}*', 'manifest.safe'))[0]
    tmp_folder = os.path.join(output_folder, 'tmp')
    os.makedirs(tmp_folder, exist_ok=True)
    phase_filename = os.path.join(tmp_folder, f'phase_coh_{pair}.dim')
    cmd_interferogram = (
        f'gpt "{graph_interferogram}" -Pprm_filename="{prm_filename}" -Psec_filename="{sec_filename}" '
        f'-PcohWinRg={cohWinRg} -PcohWinAz={cohWinAz} '
        f'-PnRgLooks={nRgLooks} -PnAzLooks={nAzLooks} '
        f'-Poutput_filename="{phase_filename}" '
        f'-c 8192M'
    )
    print(cmd_interferogram)
    os.system(cmd_interferogram)

    # Prepare the snaphu export for unwrapping
    cmd_snaphu_export = (
        f'gpt "{graph_snaphu_export}" -Pphase_filename="{phase_filename}" -Poutput_folder_snaphu="{tmp_folder}" '
    )
    os.system(cmd_snaphu_export)

    # Unwrapping with snaphu
    snaphu_conf_filename = glob.glob(os.path.join(tmp_folder, '**/snaphu.conf'))[0]
    with open(snaphu_conf_filename, 'r') as file:
        for line in file:
            if line.startswith('#'):
                line = line[1:].lstrip()  # Remove the '#' symbol and whitespaces at the beginning
                if line.startswith('snaphu'):
                    cmd_unwrapping = line.rstrip()
                    break
    work_dir = os.getcwd()
    os.chdir(os.path.dirname(snaphu_conf_filename))
    print(cmd_unwrapping)
    os.system(cmd_unwrapping)
    os.chdir(work_dir)

    # Geocode the result (interferogram, unwrapped interferogram, coherence)
    prm_date, sec_date = [datetime.strptime(date, "%Y%m%d") for date in pair.split('_')]
    phase_bandname = f'Phase_ifg_{sub_swath}_VV_{prm_date.strftime("%d%b%Y")}_{sec_date.strftime("%d%b%Y")}'
    unw_phase_bandname = f'Unw_Phase_ifg_{prm_date.strftime("%d%b%Y")}_{sec_date.strftime("%d%b%Y")}'
    coh_bandname = f'coh_{sub_swath}_VV_{prm_date.strftime("%d%b%Y")}_{sec_date.strftime("%d%b%Y")}'
    unw_phase_filename = glob.glob(os.path.join(tmp_folder, '**/UnwPhase*.hdr'))[0]
    cmd_geocode = (
        f'gpt "{graph_geocode}" -Pinterferogram_filename="{phase_filename}" -Punw_interferogram_filename="{unw_phase_filename}" '
        f'-Pphase_coh_bandnames="{phase_bandname},{unw_phase_bandname},{coh_bandname}" '
        f'-PsaveDEM="{str(saveDEM).lower()}" '
        f'-Poutput_filename="{os.path.join(output_folder, f'geocoded_interferogram_{pair}.tif')}"'
    )
    print(cmd_geocode)
    os.system(cmd_geocode)
    shutil.rmtree(tmp_folder)
    saveDEM = False
    

gpt "../graphs/interferogram_sarGeometry.xml" -Pprm_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/mintpy/S1A_SLC_20190502T173426_062083_IW3_VV_199695.SAFE/manifest.safe" -Psec_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/mintpy/S1A_SLC_20190526T173427_062083_IW3_VV_202568.SAFE/manifest.safe" -PcohWinRg=11 -PcohWinAz=3 -PnRgLooks=4 -PnAzLooks=1 -Poutput_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/mintpy/tmp/phase_coh_20190502_20190526.dim" -c 8192M


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph
....11%....22%....33%....45%....56%....67%....78%....90% done.


-- org.jblas INFO Deleting /tmp/jblas16575188000659690745/libjblas_arch_flavor.so
-- org.jblas INFO Deleting /tmp/jblas16575188000659690745/libjblas.so
-- org.jblas INFO Deleting /tmp/jblas16575188000659690745/libgfortran-5.so
-- org.jblas INFO Deleting /tmp/jblas16575188000659690745/libquadmath-0.so
-- org.jblas INFO Deleting /tmp/jblas16575188000659690745
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph
...11%...22%...33%...45%...56%...67%...78%...90% done.
snaphu -f snaphu.conf Phase_ifg_IW3_VV_02May2019_26May2019.snaphu.img 6120

snaphu v2.0.4
27 parameters input from file snaphu.conf (84 lines total)


...

Executing processing graph
...11%...22%...33%...45%...56%...67%...78%...90% done.
snaphu -f snaphu.conf Phase_ifg_IW3_VV_15Mar2019_27Mar2019.snaphu.img 6120

snaphu v2.0.4
27 parameters input from file snaphu.conf (84 lines total)


-- org.jblas INFO Deleting /tmp/jblas8124978593853064282/libjblas_arch_flavor.so
-- org.jblas INFO Deleting /tmp/jblas8124978593853064282/libjblas.so
-- org.jblas INFO Deleting /tmp/jblas8124978593853064282/libgfortran-5.so
-- org.jblas INFO Deleting /tmp/jblas8124978593853064282/libquadmath-0.so
-- org.jblas INFO Deleting /tmp/jblas8124978593853064282


Logging run-time parameters to file snaphu.log
Creating temporary directory snaphu_tiles_3228081
Unwrapping tile at row 0, column 0 (pid 3228089)
Unwrapping tile at row 0, column 1 (pid 3228092)
Unwrapping tile at row 0, column 2 (pid 3228094)
Unwrapping tile at row 0, column 3 (pid 3228103)
Unwrapping tile at row 0, column 4 (pid 3228107)
Unwrapping tile at row 0, column 5 (pid 3228112)
Unwrapping tile at row 0, column 6 (pid 3228116)
Unwrapping tile at row 0, column 7 (pid 3228120)
Unwrapping tile at row 0, column 8 (pid 3228122)
Unwrapping tile at row 0, column 9 (pid 3228131)
Unwrapping tile at row 1, column 0 (pid 3228137)
Unwrapping tile at row 1, column 1 (pid 3228138)
Unwrapping tile at row 1, column 2 (pid 3228139)
Unwrapping tile at row 1, column 3 (pid 3228140)
Unwrapping tile at row 1, column 4 (pid 3228148)
Unwrapping tile at row 1, column 5 (pid 3228149)
Unwrapping tile at row 1, column 6 (pid 3228153)
Unwrapping tile at row 1, column 7 (pid 3228158)
Unwrapping tile at ro

65 incremental costs clipped to avoid overflow (0.002%)
66 incremental costs clipped to avoid overflow (0.002%)
67 incremental costs clipped to avoid overflow (0.002%)
67 incremental costs clipped to avoid overflow (0.002%)
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph
....10%....21%....32%....42%....53%....64%....75%....85%.. done.


## Format the data for MintPy

In [4]:
mintpy_folder = os.path.join(output_folder, 'mintpy')
os.makedirs(mintpy_folder)
os.makedirs(os.path.join(mintpy_folder, 'interferograms'))
for filename in glob.glob(f'{os.path.join(output_folder, "*.tif")}'):

    date_pair = os.path.basename(filename).replace("geocoded_interferogram_", "")[:-4]
    folder_interferogram = os.path.join(mintpy_folder, 'interferograms', date_pair)
    os.makedirs(folder_interferogram)

    metadata = subprocess.run(
        ["tiffinfo", filename],
        stdout=subprocess.PIPE,  # capture standard output
        stderr=subprocess.PIPE,  # capture warnings/errors if needed
        text=True                # decode bytes to str
    ).stdout

    band_names = re.findall(r"<BAND_NAME>(.*?)</BAND_NAME>", metadata)

    for b in band_names:

        if b.startswith("Phase"):
            output_filename = os.path.join(folder_interferogram, f'{date_pair}_filt_int_sub_tc')
        elif b.startswith("Unw"):
            output_filename = os.path.join(folder_interferogram, f'{date_pair}_unw_tc')
        elif b.startswith("coh"):
            output_filename = os.path.join(folder_interferogram, f'{date_pair}_coh_tc')
        elif b == 'elevation':
            output_filename = os.path.join(mintpy_folder, 'dem_tc')

        cmd_subset = (
            f'gpt "{graph_subset}" -Pinput_filename="{filename}" '
            f'-Pband_name="{b}" -Pregion="{aoi}" '
            f'-Poutput_filename="{output_filename}"'
        )
        os.system(cmd_subset)

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph
.16%..32%.49%.65%.81%. done.


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph
.16%..32%.49%.65%.81%. done.


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


...

Executing processing graph
.16%..32%.49%.65%.81%. done.


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph
.16%..32%.49%.65%.81%. done.


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph
.16%..32%.49%.65%.81%. done.


## Create the MintPy config file

Define reference latitude and longitude:

In [5]:
ref_lat = -33
ref_lon = 18.05

In [6]:
config_filename = os.path.join(mintpy_folder, 'mintpy_config.txt')

unw_files = os.path.join(mintpy_folder, 'interferograms', '*', '*', 'Unw_*.img')
coh_files = os.path.join(mintpy_folder, 'interferograms', '*', '*', 'coh_*.img')
dem_file  = os.path.join(mintpy_folder, 'dem_tc.data', 'elevation.img')

config_txt = f"""
mintpy.load.processor   = snap
mintpy.load.unwFile     = {unw_files}
mintpy.load.corFile     = {coh_files}
mintpy.load.demFile     = {dem_file}

mintpy.reference.lalo   = {ref_lat}, {ref_lon}

mintpy.networkInversion.weightFunc              = var
mintpy.deramp                                   = linear
mintpy.topographicResidual.pixelwiseGeometry    = no
"""

with open(config_filename, "w") as file:
    file.write(config_txt)

## Run Mintpy

In [7]:
cmd_mintpy = f'smallbaselineApp.py {config_filename}'
print(cmd_mintpy)
os.chdir(mintpy_folder)
os.system(cmd_mintpy)

smallbaselineApp.py /home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/mintpy/mintpy/mintpy_config.txt

___________________________________________________________

  /##      /## /##             /##     /#######
 | ###    /###|__/            | ##    | ##__  ##
 | ####  /#### /## /#######  /######  | ##  \ ## /##   /##
 | ## ##/## ##| ##| ##__  ##|_  ##_/  | #######/| ##  | ##
 | ##  ###| ##| ##| ##  \ ##  | ##    | ##____/ | ##  | ##
 | ##\  # | ##| ##| ##  | ##  | ## /##| ##      | ##  | ##
 | ## \/  | ##| ##| ##  | ##  |  ####/| ##      |  #######
 |__/     |__/|__/|__/  |__/   \___/  |__/       \____  ##
                                                 /##  | ##
                                                |  ######/
   Miami InSAR Time-series software in Python    \______/
          MintPy 1.6.2, 2025-07-07
___________________________________________________________

--RUN-at-2025-12-09 09:02:33.560487--
Current directory: /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy
Run

/home/mcallegari/anaconda3/envs/cloudinsar/lib/python3.13/site-packages/mintpy/utils/utils1.py:198: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=50000`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.islice`.
Please see the 1.23 release notes for an example on how to do this.  If you wish to ignore this warning, use `warnings.filterwarnings`.  This warning is expected to be removed in the future and is given only once per `loadtxt` call.
  txtContent = np.loadtxt(fname, dtype=bytes).astype(str)


 
write average value in space into text file: coherenceSpatialAvg.txt
number of acquisitions: 10
number of interferograms: 21
shift all perp baseline by 81.36494445800781 to zero mean for plotting
--------------------------------------------------
number of interferograms marked as drop: 0
number of interferograms marked as keep: 21
number of acquisitions marked as drop: 0
save figure to pbaseHistory.pdf
save figure to coherenceMatrix.pdf
save figure to coherenceHistory.pdf
max perpendicular baseline: 168.00 m
max temporal      baseline: 36.0 days
showing coherence
data range: [0.3713, 0.5413]
display range: [0.2, 1.0]
save figure to network.pdf


******************** step - reference_point ********************
Input data seems to be geocoded. Lookup file not needed.

generate_mask.py /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/inputs/ifgramStack.h5 --nonzero -o /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/maskConnComp.h5 --update
input ifgramStack file: /mnt/CEPH_PRO

/home/mcallegari/anaconda3/envs/cloudinsar/lib/python3.13/site-packages/mintpy/objects/stack.py:1059: RuntimeWarning: Mean of empty slice
  dmean[r0:r1, :] = np.nanmean(data, axis=0)
<frozen genericpath>:39: RuntimeWarning: bool is used as a file descriptor


 
create HDF5 file: /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/avgSpatialCoh.h5 with w mode
create dataset /coherence of float32    in size of (613, 1559)          with compression=None
finished writing to /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/avgSpatialCoh.h5
time used: 00 mins 0.1 secs

Input data seems to be geocoded. Lookup file not needed.

reference_point.py /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/inputs/ifgramStack.h5 -t /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/smallbaselineApp.cfg -c /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/avgSpatialCoh.h5
--------------------------------------------------
reading reference info from template: /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/smallbaselineApp.cfg
input reference point in lat/lon: (-33.0, 18.05)
input reference point in y/x: (222, 167)
mask: maskConnComp.h5
--------------------------------------------------
calculate the temporal average of unwrapPhase in file /mn

/home/mcallegari/anaconda3/envs/cloudinsar/lib/python3.13/site-packages/mintpy/utils/utils1.py:150: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=50000`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.islice`.
Please see the 1.23 release notes for an example on how to do this.  If you wish to ignore this warning, use `warnings.filterwarnings`.  This warning is expected to be removed in the future and is given only once per `loadtxt` call.
  fc = np.loadtxt(rms_file, dtype=bytes).astype(str)
/home/mcallegari/anaconda3/envs/cloudinsar/lib/python3.13/site-packages/mintpy/utils/plot.py:1029: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=50000`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.

 
save timeseries RMS to text file: /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/rms_timeseriesResidual_ramp.txt
read timeseries residual RMS from file: /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/rms_timeseriesResidual_ramp.txt
--------------------------------------------------
date with min RMS: 20190408 - 0.0017
save date to file: reference_date.txt
--------------------------------------------------
date(s) with RMS > 3.0 * median RMS (0.0100)
None.
save figure to file: /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/rms_timeseriesResidual_ramp.pdf


******************** step - reference_date ********************

reference_date.py -t /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/smallbaselineApp.cfg /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/timeseries.h5 /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/timeseries_ERA5.h5 /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/timeseries_ERA5_ramp.h5 /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/min

/home/mcallegari/anaconda3/envs/cloudinsar/lib/python3.13/site-packages/mintpy/view.py:1366: UserWarning: Ignoring specified arguments in this call because figure with num: 1 already exists
  fig, axs = plt.subplots(
/home/mcallegari/anaconda3/envs/cloudinsar/lib/python3.13/site-packages/mintpy/view.py:1366: UserWarning: Ignoring specified arguments in this call because figure with num: 1 already exists
  fig, axs = plt.subplots(
/home/mcallegari/anaconda3/envs/cloudinsar/lib/python3.13/site-packages/mintpy/view.py:1366: UserWarning: Ignoring specified arguments in this call because figure with num: 1 already exists
  fig, axs = plt.subplots(



--------------------------------------------------
using all dates to calculate the time function
--------------------------------------------------
estimate deformation model with the following assumed time functions:
    polynomial : 1
    periodic   : []
    stepDate   : []
    polyline   : []
    exp        : {}
    log        : {}
add/update the following configuration metadata:
['startDate', 'endDate', 'excludeDate', 'polynomial', 'periodic', 'stepDate', 'exp', 'log', 'uncertaintyQuantification', 'timeSeriesCovFile', 'bootstrapCount']
--------------------------------------------------
create HDF5 file: /mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Test/mintpy/mintpy/velocityERA5.h5 with w mode
create dataset  : intercept    of <class 'numpy.float32'>   in size of (613, 1559)          with compression = None
create dataset  : interceptStd of <class 'numpy.float32'>   in size of (613, 1559)          with compression = None
create dataset  : velocity     of <class 'numpy.float32'>   in size of

0

In [ ]:
os.chdir(work_dir)

In [12]:
# import rasterio

# for img in glob.glob(unw_files):
#     with rasterio.open(img) as src:
#         width = src.width
#         height = src.height
#         count = src.count   # number of bands
        
#         print(f"{width} x {height}, {count} bands")

1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
1559 x 613, 1 bands
